# Lecture 0 - Introduction to QuTiP

Author: J. R. Johansson (robert@riken.jp), https://jrjohansson.github.io/

This lecture series was developed by J.R. Johannson. The original lecture notebooks are available [here](https://github.com/jrjohansson/qutip-lectures).

This is a slightly modified version of the lectures, to work with the current release of QuTiP. You can find these lectures as a part of the [qutip-tutorials repository](https://github.com/qutip/qutip-tutorials). This lecture and other tutorial notebooks are indexed at the [QuTiP Tutorial webpage](https://qutip.org/tutorials.html).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image
from qutip import (Qobj, about, basis, coherent, coherent_dm, create, destroy,
                   expect, fock, fock_dm, mesolve, qeye, sigmax, sigmay,
                   sigmaz, tensor, thermal_dm, anim_matrix_histogram,
                   anim_fock_distribution)
# set a parameter to see animations in line
from matplotlib import rc
rc('animation', html='jshtml')

%matplotlib inline

## Introduction

QuTiP is a python package for calculations and numerical simulations of quantum systems.

It includes facilities for representing and doing calculations with quantum objects such state vectors (wavefunctions), as bras/kets/density matrices, quantum operators of single and composite systems, and superoperators (useful for defining master equations).

It also includes **solvers for a time-evolution of quantum systems, according to: Schrodinger equation, von Neuman equation, master equations, Floquet formalism, Monte-Carlo quantum trajectors, experimental implementations of the stochastic Schrodinger/master equations.**

For more information see the project web site at [qutip.org](https://qutip.org), and the
[QuTiP documentation](https://qutip.readthedocs.io/en/latest/index.html).

### Installation

You can install QuTiP directly from `pip` by running:

`pip install qutip`

For further installation details, refer to the [GitHub repository](https://github.com/qutip/qutip).

## Quantum object class: `qobj`

At the heart of the QuTiP package is the `Qobj` class, which is used for *representing quantum object such as states and operator.*

The `Qobj` class contains all the information required to describe a quantum system, such as *its matrix representation, composite structure and dimensionality.*

In [ ]:
Image(filename="images/qobj.png")

### Creating and inspecting quantum objects

We can create a new quantum object using the `Qobj` class constructor, like this:

In [ ]:
q = Qobj([[1], [0]])

q

Here we passed python list as an argument to the class constructor. The data in this list is used to construct the matrix representation of the quantum objects, and the other properties of the quantum object is by default computed from the same data.

We can inspect the properties of a `Qobj` instance using the following class method:

In [ ]:
# the dimension, or composite Hilbert state space structure
q.dims

In [ ]:
# the shape of the matrix data representation
q.shape

In [ ]:
# the matrix data itself. in sparse matrix format.
q.data


`fortran=True` 参数表示该 **稠密矩阵在内存中是按 Fortran（列优先）顺序排列的。**

* 背景解释：
    * Python 默认的 NumPy 数组是 row-major（行优先），也叫 C-order。

    * Fortran-order（列优先）：数据在内存中是按列排列的。

    * 在科学计算、线性代数、稀疏矩阵处理（如 QuTiP）中，**Fortran-order 对性能优化非常关键**，尤其当底层库（如 LAPACK、BLAS）使用 Fortran-style 结构。

* 在 QuTiP 中的用途：
    * QuTiP 经常调用底层的 Fortran 库来**加速线性代数运算（如乘法、特征值求解）**，因此用 `fortran=True` 可以：
        * 提高矩阵运算的速度；
        * 与 Fortran 接口保持兼容性；
        * 避免不必要的内存复制。

In [ ]:
# get the dense matrix representation
q.full()

* **“dense matrix representation” 是什么意思？**
    * 这是指将一个 量子对象 q（通常是稀疏矩阵）转化为一个完整的二维数组（NumPy array）形式，其中所有元素（包括0）都明确存储出来。

*  **在 QuTiP 中的用途**：
    * q.full() 常用于 **调试、可视化**，因为稀疏格式不易阅读；
    * 也用于 **与不支持稀疏结构的库（如纯 NumPy）交互**。

In [ ]:
# some additional properties
q.isherm, q.type

1. `q.isherm`
* 表示 `q` 是否是 **Hermitian（厄米）算符**：
    * Hermitian：矩阵满足 `q == q.dag()`（共轭转置后相等）；
    * 物理意义：Hermitian 矩阵在量子力学中代表**可观测量**（如哈密顿量、测量算符）。

* `False` 表示：
    * 这个量子对象 **不是 Hermitian**，因此不能作为测量算符或 Hamiltonian。



2. `q.type`
* 表示 `q`的类型，`ket`说明这是一个 **态矢量（Ket vector）**

### Using `Qobj` instances for calculations

With `Qobj` instances we can do arithmetic and apply a number of different operations using class methods:

In [ ]:
sy = Qobj([[0, -1j], [1j, 0]])  # the sigma-y Pauli operator

sy

1. **什么是 Pauli Operator（泡利算符）？**
* Pauli 算符是三个 $2 \times 2$ 的厄米矩阵，分别表示：

| 名称      | 符号            | 矩阵表示形式                                           |
| ------- | ------------- | ------------------------------------------------ |
| Pauli-X | $\sigma_x$ | $\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$  |
| Pauli-Y | $\sigma_y$ | $\begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}$ |
| Pauli-Z | $\sigma_z$ | $\begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$ |

它们作用在 **单量子比特的希尔伯特空间上**，满足如下代数结构：
        $$
        [\sigma_i, \sigma_j] = 2i \varepsilon_{ijk} \sigma_k
        $$

2.  **Pauli-Y 有什么用？**
* 在量子信息中的主要用途：

| 应用场景           | 作用                                       |
| -------------- | ---------------------------------------- |
| 量子门            | $\sigma_y$ 本身就是一个基本的**量子逻辑门（Y-gate）** |
| 态的旋转           | 可用于旋转 qubit 的 Bloch 球上的状态向量              |
| 量子纠缠运算         | 和其他 Pauli 门组合可用于构造 Bell 态等               |
| Hamiltonian 构建 | 常用于定义自旋系统的哈密顿量，如 Ising、Heisenberg 模型     |


* 具体是：
    * 对量子态的变化：
    $$
    \sigma_y |0\rangle = i|1\rangle,\quad
    \sigma_y |1\rangle = -i|0\rangle
    $$
    * 与 Pauli-X 类似，但多了一个 **复数相位因子**（$±i$）！

* **所以它的本质作用是**：

    * > **翻转 $\left|0\right\rangle$ 与 $\left|1\right\rangle$，同时施加一个相位旋转 $±i$。**

In [ ]:
sz = Qobj([[1, 0], [0, -1]])  # the sigma-z Pauli operator

sz

###  Pauli-Z ($\sigma_z$) — 相位翻转（Phase flip）

---

#### 作⽤：

* 它保持 $\left|0\right\rangle$ 不变，改变 $\left|1\right\rangle$ 的符号：
$$
\sigma_z \left|0\right\rangle = \left|0\right\rangle,\quad
\sigma_z \left|1\right\rangle = -\left|1\right\rangle
$$

* 在 Bloch 球上：**绕 Z 轴旋转 π 弧度**

---

#### 应用场景：

| 用途         | 说明                        |
| ---------- | ------------------------- |
| 相位调控       | 只改变量子态相位而不影响振幅            |
| 用于纠错码      | 可检测/修复 Phase flip 错误      |
| Z basis 测量 | 是 Z 方向测量的本征算符（observable） |

---


#### **Pauli-Z（σ_z）确实用于 Grover 算法中的“相位翻转（phase inversion）”**，这正是它的关键作用之一！
>  **Pauli-Z 是 Grover 算法实现 oracle 的简洁工具**，特别适合当目标态是 $|1\rangle$。

---

####  回顾 Grover 算法中的两大步骤：

1. **标记目标态（oracle）**
   对目标态加负号，即：

   $$
   |x\rangle \to -|x\rangle \quad \text{（仅当 } x \text{ 是目标解）}
   $$

2. **扩散变换（diffusion operator）**
   把目标态的相位翻转后，再进行整体放大。

---

####  Pauli-Z 如何用于 Grover Oracle？

设目标态是 $|1\rangle$，用 $\sigma_z$ 作用：

$$
\sigma_z |0\rangle = |0\rangle,\quad
\sigma_z |1\rangle = -|1\rangle
$$

这恰好就是 **只对目标态加负号，其他态保持不变** —— 完美实现 oracle！

---

#### 更通用的做法：

Grover oracle 通常写成：

$$
O_f = I - 2 |x\rangle\langle x|
$$

但对于单比特时，直接用 Pauli-Z 就足够表达 oracle 功能。

---



### Pauli-X ($\sigma_x$) — 类似经典的 “NOT” 门

---

#### 作⽤：

* 它把 $\left|0\right\rangle$ 和 $\left|1\right\rangle$ 状态互换：
$$
\sigma_x \left|0\right\rangle = \left|1\right\rangle,\quad
\sigma_x \left|1\right\rangle = \left|0\right\rangle
$$

* 在 Bloch 球上：**绕 X 轴旋转 π 弧度**

---

#### 应用场景：

| 用途     | 说明                          |
| ------ | --------------------------- |
| 量子翻转   | 相当于经典的 bit flip（0 ↔ 1）      |
| 构造量子电路 | 基本单量子比特门                    |
| 纠缠构建   | 配合 Hadamard、CNOT 构造 Bell 态等 |

In [ ]:
# some arithmetic with quantum objects

H = 1.0 * sz + 0.1 * sy

print("Qubit Hamiltonian = \n")
H

Example of modifying quantum objects using the `Qobj` methods:

In [ ]:
# Compute the hermitian conjugate of `sy`
sy.dag()

In [ ]:
# The trace
H.tr()

#### `H.tr()`是求 **Hamiltonian（哈密顿算符）`H` 的 trace（迹）**。
* 所以图里 `H.tr()` 是：
    * 求哈密顿算符的对角线之和；
    * 结果是 `0.0`，说明：
        * $H$ 的本征值加和为零；
        * 它可能代表一个 **中心对称能级结构**（如：对称双能级系统）

* 在量子物理中，它关联：
    * 能量谱结构
    * 密度矩阵规范性
    * 可观测量期望计算

---

####  什么是 **Trace（迹）**？

> **矩阵的迹是对角线元素的总和。**

* 数学表达：

    $$
    \text{Tr}(A) = \sum_i A_{ii}
    $$

* 例如：

    $$
    A = \begin{pmatrix}
    1 & 2 \\
    3 & 4
    \end{pmatrix}
    \quad \Rightarrow \quad \text{Tr}(A) = 1 + 4 = 5
    $$

---

#### 在量子力学中的意义？

| Trace 的对象                    | 物理含义                                     |
| ---------------------------- | ---------------------------------------- |
| $\text{Tr}(\rho)$          | 密度矩阵的总概率（必须为 1）                          |
| $\text{Tr}(H)$             | 哈密顿量的能量本征值之和                             |
| $\text{Tr}(\rho A)$        | 态 $\rho$ 中可观测量 $A$ 的期望值              |
| $\text{Tr}_{B}(\rho_{AB})$ | 对子系统 $B$ 做部分迹，得到 $A$ 的 reduced state |

---

#### 关于 **“对子系统 $B$ 做部分迹，得到 $A$ 的 reduced state”**。
> **部分迹** 是从复合系统中“忽略”掉一部分，只保留感兴趣部分的量子态，是量子系统**局部行为研究**的核心工具。

* #### 清晰表达（改进 Markdown 版）

> **$\mathrm{Tr}_B(\rho_{AB})$**
> 表示：对复合系统 $AB$ 的密度矩阵 $\rho_{AB}$，在 **子系统 $B$ 上取迹**，得到 **子系统 $A$ 的约化密度矩阵**（reduced density matrix）：
>
> $$
> \rho_A = \mathrm{Tr}_B(\rho_{AB})
> $$

---

* #### 物理含义（为什么要部分迹）

    * 当你只**关心系统 $A$ 的行为**，不关心 $B$，就**对 $B$ 做 trace**。
    * 它等价于：**“忽略”$B$，在 $A$ 上保留统计信息**。
    * 类似于统计学中的 **边缘分布**。

---

* #### 示例场景

    * 假设系统是两个 qubit（$A$ 和 $B$）组成的 Bell 态：

        $$
        |\Psi\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)
        \Rightarrow \rho_{AB} = |\Psi\rangle\langle\Psi|
        $$

    * 这个是一个 $4 \times 4$ 的密度矩阵。

    * 我们现在只想看子系统 $A$ 的状态 → 做 **partial trace over B**。

---


#### QuTiP 实例代码

In [ ]:
from qutip import basis, tensor, ket2dm, ptrace

# 定义 Bell 态
psi = (tensor(basis(2, 0), basis(2, 0)) + tensor(basis(2, 1), basis(2, 1))).unit()
rho_AB = ket2dm(psi)  # 构造 rho = |psi><psi|

# 对 B 做部分迹，得到 rho_A
rho_A = ptrace(rho_AB, 0)

# 打印 rho_A
print(rho_A)

输出是：

$$
\rho_A = 
\begin{pmatrix}
0.5 & 0 \\
0 & 0.5
\end{pmatrix}
\Rightarrow \text{maximally mixed state}
$$

说明：**虽然整体是纯态，但 $A$ 单独来看是混态**。

#### trace一定是左上到右下的对角线吗？有什么原因吗？左下到右上呢？

* 是的，**trace（迹）就是左上到右下的主对角线** 元素之和。
    * 即：

    $$
    \text{Tr}(A) = \sum_{i=1}^n A_{ii}
    $$

    * 其中 $A_{ii}$ 表示第 $i$ 行第 $i$ 列的元素。

* **trace 一定是主对角线（左上 → 右下）**，因为这是唯一一个：
    >
    > * 能与本征值匹配
    > * 保持不变量结构
    > * 有物理和数学意义的结构

---

* 右上到左下（副对角线）**不是 trace**

    * 这条对角线不涉及任何标准定义中的迹，它没有特殊的数学含义，**除非某些特定应用人为定义它**。
    * 它不是线性变换的“投影”；
    * 无法推导出本征值、期望值、归一性等关键量；
    * **没有不变量性质**（对 unitary 变换会变）

---

#### 为什么主对角线是 trace？

数学上的原因：

1. **Trace 是矩阵与单位矩阵的 Frobenius 内积**：

$$
\text{Tr}(A) = \langle A, I \rangle = \sum_{i,j} A_{ij} I_{ij} = \sum_i A_{ii}
$$

2. **与线性变换本征值有关**：

$$
\text{Tr}(A) = \sum_i \lambda_i \quad \text{（所有本征值之和）}
$$

主对角线正好是本征值分布的投影结果。

---

#### Trace 的核心性质：

| 性质                                          | 描述                           |
| ------------------------------------------- | ---------------------------- |
| $\text{Tr}(AB) = \text{Tr}(BA)$           | 可循环移位（但不交换）                  |
| $\text{Tr}(U^\dagger A U) = \text{Tr}(A)$ | 单位变换下保持不变（unitary invariant） |
| $\text{Tr}(\rho) = 1$                     | 量子态密度矩阵的规范条件                 |

这些性质都与主对角线的定义相关，**和副对角线无关**。


In [ ]:
# Eigen energies
H.eigenenergies()

In [ ]:
# Eigen states
H.eigenstates()  # 返回 (energies, states)

`H.eigenstates()`计算的是：**Hamiltonian $H$ 的本征能量（Eigenenergies）**

> * 本征能量（Eigenenergies） = 哈密顿算符的本征值 = 系统的可能能量测量结果
> * 即，这个系统能“处于”的能量层级。


---

#### 什么是本征能量（Eigenenergies）？

在量子力学中，**哈密顿量 $H$ 表示一个量子系统的总能量**。

求它的本征值问题：

$$
H |\psi_n\rangle = E_n |\psi_n\rangle
$$

这里的 $E_n$ 就是：

> ✅ **该系统可能的能量值（即本征能量）**

而 $|\psi_n\rangle$ 是对应的本征态（量子态）。

---

#### 回到这里的print：

```python
array([-1.00498756,  1.00498756])
```

这表示：

* 该系统有两个能量本征值（说明 $H$ 是 $2\times2$）
* 能量谱是对称的：$±1.00498756$
* 对应的是两个本征态（态矢量）

---

#### 实际物理意义

* 本征能量 $E_n$ 是系统在测量时可能得到的能量值；
* 演化行为由这些 $E_n$ 和 $|\psi_n\rangle$ 决定；
* 在时间演化中使用：

$$
|\psi(t)\rangle = \sum_n e^{-i E_n t} c_n |\psi_n\rangle
$$

---

### `eigen + xxx`
* #### eigenvalues
* #### eigenvectors
* #### eigenenergies
* #### eigenstates
> #### **所有 `eigen+xxx` 都源自同一套特征值问题，只是在数学或量子语境中强调不同侧重点。**

---

#### 它们都源自同一个数学问题：

> 对于一个算符（或矩阵）$A$，求解：
>
> $$
> A \mathbf{v} = \lambda \mathbf{v}
> $$
>
> 其中：
>
> * $\lambda$ 是 **本征值（eigenvalue）**
> * $\mathbf{v}$ 是 **本征向量（eigenvector）**

---

#### 分类总结表

| 术语              | 含义                               | 常用于   |
| --------------- | -------------------------------- | ----- |
| `eigenvalues`   | \$\lambda\$，本征值：**系统可能测量出的数值结果** | 数学、物理 |
| `eigenvectors`  | \$\mathbf{v}\$，对应的向量             | 数学    |
| `eigenenergies` | Hamiltonian 的本征值，表示**能量本征值**     | 量子物理  |
| `eigenstates`   | Hamiltonian 的本征态（即 ket 矢量）       | 量子物理  |

---

#### 举例说明（量子背景）

以量子哈密顿量 $H$ 为例：

$$
H |\psi_n\rangle = E_n |\psi_n\rangle
$$

| 符号                   | QuTiP 方法               | 含义                                              |
| -------------------- | ---------------------- | ---------------------------------------------- |
| $E_n$             | `H.eigenenergies()`    | 本征能量（能量本征值）                                     |
| $\psi_n\rangle$                   | `H.eigenstates()`       |  本征态（以 Qobj 给出的 ket） |
| numpy 中的 $\lambda$ | `np.linalg.eigvals(H)` | 纯数学求本征值                                          |
| numpy 中的 $v$       | `np.linalg.eig(H)`     | 返回 `(eigenvals, eigenvecs)`                     |

---

#### 更进一步理解：
* 数学角度关注 **“这个矩阵有啥线性结构？”**

* 物理角度关注 **“这个系统可能处在哪种状态、能量是多少？”**

它们是**同一组对象在不同语境下的命名与解释**。

---

#### 关键区别：

| `eigenvectors`（NumPy） | `eigenstates`（QuTiP） |
| --------------------- | -------------------- |
| 返回的是 **数组**（列向量）      | 返回的是 **Qobj 量子态**    |
| 没有物理意义（纯数学）           | 是 ket 向量，有物理意义       |

---


In [ ]:
# eigvals, eigvecs = np.linalg.eig(H)
# 上行报错是因为直接把 QuTiP 的 Qobj 喂给了 NumPy 的线性代数函数。
# 解决办法是加 .full() 转换为 2D NumPy 矩阵即可。

# H.full() 会把 H 从 Qobj（稀疏/量子结构）转为普通的 np.ndarray
# 之后 np.linalg.eig() 就可以正常工作了
# 因为 np.linalg.eig() 要求输入是 np.ndarray，但 Qobj 不是。

eigvals, eigvecs = np.linalg.eig(H.full())
print(eigvals)
print(eigvecs)

In [ ]:
energies, states = H.eigenstates()
print(energies)
print(states)


In [ ]:
print(H)

For a complete list of methods and properties of the `Qobj` class, see the [QuTiP documentation](https://qutip.readthedocs.io/en/latest/index.html) or try `help(Qobj)` or `dir(Qobj)`.

In [ ]:
help(Qobj)

In [ ]:
dir(Qobj)

## States and operators

Normally we do not need to create `Qobj` instances from stratch, using its constructor and passing its matrix represantation as argument. Instead we can use functions in QuTiP that generates common states and operators for us. Here are some examples of built-in state functions:

### State vectors

In [ ]:
# Fundamental basis states (Fock states of oscillator modes)

N = 2  # number of states in the Hilbert space
n = 1  # the state that will be occupied

basis(N, n)  # equivalent to fock(N, n)

#### 什么是 Fock state？

>  **Fock 态** 是 **粒子数已知的量子态**，也称为 **number states**。

用 Dirac 表示法：

$$
|n\rangle
$$

表示系统中有 $n$ 个激发（例如光子、粒子、量子）。

---

#### 和“oscillator modes”什么关系？

在量子谐振子（harmonic oscillator）中，每个模式（mode）有一套 Fock 态基底：

$$
|0\rangle, |1\rangle, |2\rangle, \dots
$$

每个态代表这个模式里有多少个量子（能量单位）。

---

#### 这里构造的是：

$$
|1\rangle = \begin{pmatrix} 0 \\ 1 \end{pmatrix}
$$

也就是**2维希尔伯特空间中的第 1 个 Fock 态** *（注意索引从 0 开始）*。


In [ ]:
fock(4, 2)  # another example of set up 
            # a fundamental basis states (Fock states of oscillator modes)

返回一个 $4 \times 1$ 的 **Fock 态**（也就是 ket 向量）：

$$
|2\rangle = 
\begin{pmatrix}
0 \\
0 \\
1 \\
0
\end{pmatrix}
$$

表示：**在 4 维希尔伯特空间中，只有第 3 项（索引 2）被激发，其余为 0。**

---

#### 小区别（本质相同）：

| 函数        | 用途/偏好          |
| --------- | -------------- |
| `basis()` | 更通用的“数学基底”生成器  |
| `fock()`  | 强调“量子态中的粒子数表征” |

也就是说：

* `basis()` → 从线性代数角度命名
* `fock()`  → 从量子物理语义命名

但本质都是构造稀疏向量：

```python
assert fock(4, 2) == basis(4, 2)
```

In [ ]:
# a coherent state
coherent(N=10, alpha=1.0)

#### 什么是 coherent state（相干态）？

> 相干态是谐振子（如电磁场模式）中最接近**经典振荡**的量子态。

它是 **调和振子**的最小不确定性解，形式为：

$$
|\alpha\rangle = e^{-|\alpha|^2/2} \sum_{n=0}^{\infty} \frac{\alpha^n}{\sqrt{n!}} |n\rangle
$$

其中：

* $\alpha$ 是复振幅（决定振荡强度和相位）
* $|n\rangle$ 是 Fock（数）态
* 总体结构是一个 **泊松分布** 的叠加态

---

#### 这里代码输出的意义：

```python
coherent(N=10, alpha=1.0)
```

→ 在 $N=10$ 的截断希尔伯特空间内生成：

$$
|\alpha=1\rangle \approx 0.607 |0\rangle + 0.607 |1\rangle + 0.429 |2\rangle + \dots
$$

表示：**每个粒子数态都有一定概率被激发**，符合经典振荡器的行为！

---

#### 应用领域

| 场景               | 作用                   |
| ---------------- | -------------------- |
| 量子光学             | 表示激光态、腔模态等           |
| QED / cavity QED | 模拟光场与原子的耦合           |
| 相干态控制            | 用于最小不确定性的态制备         |
| 量子计算模拟           | 某些连续变量量子信息处理中作为输入参考态 |

#### QuTiP 中常用操作
下个cell的代码会画出 **Fock 态概率分布图**（泊松型）。

In [ ]:
from qutip import coherent, plot_fock_distribution

state = coherent(N=10, alpha=1.0)
plot_fock_distribution(state)

### Density matrices

In [ ]:
# a fock state as density matrix
fock_dm(5, 2)  # 5 = hilbert space size, 2 = state that is occupied

#### 上个cell的结果只有 $\rho_{22}$ 为 1（注意从 0 开始编号，这是 Python 惯例），其余为 0。
> 只有 $\rho_{22} = 1$ 是因为你创建的是 $|2\rangle\langle 2|$ 的密度矩阵，它只“占据”第 2 个 Fock 态，对角线上表示态的概率，非对角项表示相干性。


问：**为啥不是每列第 2 个元素是 1？**

---

##### 核心解释：这是外积 $|2\rangle\langle2|$ 的结果

* 原始态：basis(5, 2), 表示：
         $$
        |2\rangle =
        \begin{pmatrix}
        0 \\
        0 \\
        1 \\
        0 \\
        0
        \end{pmatrix}
        $$

* 而密度矩阵构造方式：

$$
\rho = |2\rangle \langle 2| =
\begin{pmatrix}
0 \\
0 \\
1 \\
0 \\
0
\end{pmatrix}
\cdot
\begin{pmatrix}
0 & 0 & 1 & 0 & 0
\end{pmatrix}
=
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0
\end{pmatrix}
$$

> 所以只有中间那一项是 1，表示系统 **100% 处于** $|2\rangle$ 态。

---


In [ ]:
# coherent state as density matrix
coherent_dm(N=8, alpha=1.0)

#### 什么是 **density matrix（密度矩阵）**？

> **密度矩阵 $\rho$ 是描述量子系统状态的另一种方式，尤其适用于混态（mixed state）和统计系统。**<br>
> **密度矩阵是量子态的“全貌”，适用于纯态、混态、测量、退相干、部分迹、期望值计算等一切任务，是比 $|\psi\rangle$ 更通用的状态表示方式。**

---

#### 数学定义：

* 纯态（pure state）：

    * 当系统处于某个态 $|\psi\rangle$，密度矩阵定义为：

    $$
    \rho = |\psi\rangle \langle \psi|
    $$

    * 这是一个**投影算符**，形如外积，维度为 $N \times N$。

---

##### 区别总结表：

| 对比项      | 内积 ⟨ψ∣ϕ⟩            | 外积 ∣ψ⟩⟨ϕ∣           |                               
| -------- | ------------------- | ------------------- | 
| **定义**   | 行向量与列向量的乘积          | 列向量与行向量的乘积          |                                
| **结果类型** | 一个**标量**（complex 数） | 一个**矩阵/算符**（可施加于状态） |                                
| **维度**   | 1×n 与 n×1 → 1×1（标量） | n×1 与 1×n → n×n（矩阵） |                                
| **用途**   | 计算**概率幅**、正交性       | 构造**投影算符**、密度矩阵     |         


* **示例**
    * $⟨0|1⟩$ = 0（正交）  
    * $|0\rangle \langle 0| =\begin{bmatrix} 1 \\ 0 \end{bmatrix}\begin{bmatrix} 1 & 0 \end{bmatrix}=\begin{bmatrix}1 & 0 \\0 & 0\end{bmatrix}$
---

##### 图示理解：

* **内积**：

  $$
  \langle \psi | \phi \rangle = \sum_i \psi_i^* \phi_i \quad \text{（得到一个复数）}
  $$

* **外积**：

  $$
  | \psi \rangle \langle \phi | = 
  \begin{bmatrix}
  \psi_1 \\
  \psi_2 \\
  \vdots
  \end{bmatrix}
  \begin{bmatrix}
  \phi_1^* & \phi_2^* & \cdots
  \end{bmatrix}
  = \text{一个算符/矩阵}
  $$

---

##### 应用举例：

1. **内积**：用来计算：

   * 态之间的正交性（如 `⟨0|1⟩ = 0`）
   * 测量后概率振幅 `|⟨ϕ|ψ⟩|²`

2. **外积**：

   * 构造密度矩阵 `ρ = |ψ⟩⟨ψ|`
   * 构造投影 `P = |n⟩⟨n|` 表示测量投影到态 |n⟩

---

* 混态（mixed state）：

* 当系统以概率 $p_i$ 处于若干不同的态 $|\psi_i\rangle$，定义为：

    $$
    \rho = \sum_i p_i |\psi_i\rangle \langle \psi_i|
    $$

---

#### 密度矩阵的核心作用：

| 作用             | 说明                                          |
| -------------- | ------------------------------------------- |
| 统一描述纯态与混态      | 不再区分纯量子态和经典概率混合态                            |
| 可追踪子系统状态       | 支持 partial trace，适用于多体系统                    |
| 支持统计期望计算       | $\langle A \rangle = \mathrm{Tr}(\rho A)$ |
| 反映 decoherence | 密度矩阵自然描述“退相干”现象（失去相干性）                      |

---

#### 上面两个例子：

##### 1. `fock_dm(5, 2)`

输出：

$$
\rho = |2\rangle \langle 2| =
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$$

说明：系统 **100% 处于** Fock 态 $|2\rangle$，是纯态。

---

##### 2. `coherent_dm(N=8, alpha=1.0)`

输出：一个 8×8 的稠密矩阵，非对角项 ≠ 0，说明它是：

* **纯态**（仍由某个 $|\alpha\rangle$ 构成）
* **具有相干性**（非对角项 ≠ 0 是干涉的来源）


#### ∣2⟩ 和 ⟨2∣的区别是?

> $|n\rangle$ 是列向量（态矢）<br>
> $\langle n|$ 是行向量（共轭转置）<br>
> $|n\rangle \langle n|$ 是密度矩阵（方阵）<br>
> $|nm\rangle \langle nm|$ 是两比特的密度矩阵（可张量积得到）<br>

#### 区别总结：

| 表达式                     | 类型                 | 数学意义 |                                                       
| ----------------------- | ------------------ | ---- | 
| $\boldsymbol{           2\rangle}$        | 列向量  | 态矢量（ket），表示某个**量子态**        |                        
| $\boldsymbol{\langle 2 }$                | 行向量  | 对偶矢量（bra），是 ket 的 Hermitian 共轭 |                        
| $\boldsymbol{2\rangle \langle 2  }$| 方阵   |  **密度矩阵**（或投影算符），表示一个纯态| 

具体形状：


|2⟩:        (column vector)         (5×1) <br>
⟨2|:        (row vector)            (1×5) <br>
|2⟩⟨2|:     (outer product matrix)  (5×5) <br>


---

#### 举例说明

以 $|2\rangle = [0, 0, 1, 0, 0]^T$ 为例：

$$
|2\rangle \langle 2| =
\begin{pmatrix}
0 \\
0 \\
1 \\
0 \\
0 \\
\end{pmatrix}
\begin{pmatrix}
0 & 0 & 1 & 0 & 0
\end{pmatrix}
=
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0
\end{pmatrix}
$$

---

#### 二量子比特态 $\boldsymbol{|22\rangle \langle 22|}$ 是类似的道理！

##### 可以理解成：

$$
|22\rangle = |2\rangle \otimes |2\rangle
$$

然后构造：

$$
|22\rangle \langle 22| = (|2\rangle \otimes |2\rangle)(\langle 2| \otimes \langle 2|) = |2\rangle \langle 2| \otimes |2\rangle \langle 2|
$$

这个就形成了 **一个 5×5 的密度矩阵张量积成 25×25 的密度矩阵**，仍然是纯态密度矩阵！

---

#### 用 qutip.tensor() 创建二比特系统中的纯态密度矩阵 $|22\rangle \langle 22|$
> 这是两个 5 维系统（如振荡器模式），每个处于基态 $|2\rangle$，它们的张量积构成联合系统的密度矩阵。

In [ ]:

# 单个 |2> 态
ket2 = basis(5, 2)

# 构造 |22> = |2> ⊗ |2>
ket22 = tensor(ket2, ket2)

print(ket22)

# 构造密度矩阵 rho = |22><22|
rho22 = ket22 * ket22.dag()

# 显示
rho22


In [ ]:
import matplotlib.pyplot as plt
from qutip import basis, tensor, matrix_histogram

# 构造 |22><22|
ket2 = basis(5, 2)
ket22 = tensor(ket2, ket2)
rho22 = ket22 * ket22.dag()

# 可视化
fig = plt.figure(figsize=(10, 9))
ax = fig.add_subplot(111, projection='3d')
matrix_histogram(rho22, fig=fig, ax=ax)

# 设置标签简洁化
dim = rho22.shape[0]
ticks = np.arange(0, dim, 4)  # 每隔4个 tick
labels = [f"|{i}⟩" for i in ticks]  

ax.set_xticks(ticks)
ax.set_xticklabels(labels, rotation=45, fontsize=10)
ax.set_yticks(ticks)
ax.set_yticklabels(labels, rotation=45, fontsize=10)


plt.title(r"$\rho = |22\rangle \langle 22|$")
plt.show()


In [ ]:
from qutip import basis, tensor, matrix_histogram
import matplotlib.pyplot as plt

# Create |00><00|
ket0 = basis(5, 0)
ket00 = tensor(ket0, ket0)
rho00 = ket00 * ket00.dag()

# 提取实部/虚部/模值
rho_real = rho00.full().real
rho_imag = rho00.full().imag
rho_abs = np.abs(rho00.full())

# 选绘制部分：imag / real / abs
data = rho_real   # ← 可替换成 .imag 或 np.abs(...)

# 转为 Qobj 便于绘制
rho_plot = Qobj(data)

# Visualize rho00 = |00><00|
fig = plt.figure(figsize=(10, 9))
ax = fig.add_subplot(111, projection='3d')

#  绘图
matrix_histogram(rho00, fig=fig, ax=ax)

# 设置标签简洁化
dim = rho00.shape[0]
ticks = np.arange(0, dim, 4)  # 每隔4个 tick
labels = [f"|{i}⟩" for i in ticks]  

ax.set_xticks(ticks)
ax.set_xticklabels(labels, rotation=45, fontsize=10)
ax.set_yticks(ticks)
ax.set_yticklabels(labels, rotation=45, fontsize=10)

# 添加数值注释
for x in range(dim):
    for y in range(dim):
        val = data[y, x]
        if abs(val) > 1e-4:  # 可加条件过滤极小值
            ax.text(x, y, val + 0.01, f"{val:.2f}", color='black', ha='center', va='bottom', fontsize=6)



plt.title(r"$\rho = |00\rangle \langle 00|$")
plt.show()

In [ ]:
from qutip import coherent_dm, matrix_histogram
import matplotlib.pyplot as plt

# 创建真实的 coherent 态密度矩阵（alpha=1.5）
rho_coherent = coherent_dm(N=10, alpha=1.5)

# 提取实部/虚部/模值
rho_real = rho_coherent.full().real
rho_imag = rho_coherent.full().imag
rho_abs = np.abs(rho_coherent.full())

# 选绘制部分：imag / real / abs
data = rho_real   # ← 可替换成 .imag 或 np.abs(...)

# 可视化
fig = plt.figure(figsize=(10, 9))
ax = fig.add_subplot(111, projection='3d')
matrix_histogram(rho_coherent, fig=fig, ax=ax)

dim = rho00.shape[0]
ticks = np.arange(0, dim, 4)  # 每隔4个 tick
labels = [f"|{i}⟩" for i in ticks]  


plt.title(r'Coherent State: $\rho = |\alpha\rangle \langle \alpha|$')
plt.show()


In [ ]:
from qutip import basis, coherent_dm, matrix_histogram
import matplotlib.pyplot as plt

# Fock state |2⟩⟨2|
ket2 = basis(10, 2)
rho_fock2 = ket2 * ket2.dag()

# Coherent state ρ = |α⟩⟨α|
rho_coherent = coherent_dm(N=10, alpha=1.5)

# Fock state visualization
fig1 = plt.figure(figsize=(10, 9))
ax1 = fig1.add_subplot(111, projection='3d')
matrix_histogram(rho_fock2, fig=fig1, ax=ax1)
plt.title(r'Fock State: $\rho = |2\rangle \langle 2|$')

# Coherent state visualization
fig2 = plt.figure(figsize=(10, 9))
ax2 = fig2.add_subplot(111, projection='3d')
matrix_histogram(rho_coherent, fig=fig2, ax=ax2)
plt.title(r'Coherent State: $\rho = |\alpha\rangle \langle \alpha|$, $\alpha=1.5$')

plt.show()


In [ ]:
from qutip import *
import matplotlib.pyplot as plt
from IPython.display import HTML

N = 5  # Hilbert space dimension
alpha = 1.5
a = destroy(N)

# Hamiltonian: H = a†a (number operator)
H = a.dag() * a

# initial state: coherent state
psi0 = coherent(N, alpha)
rho0 = ket2dm(psi0)

# time evolution
tlist = np.linspace(0, 2 * np.pi, 30)
result = mesolve(H, rho0, tlist, [], [])

# animate density matrix over time
sparse_states = result.states[::3]

# 动画展示
fig, anim = anim_matrix_histogram(sparse_states, limits=[-1, 1], colorbar=True)
html_anim = HTML(anim.to_jshtml())  # 保留引用
plt.close(fig)                      # 只关闭刚刚创建的fig，避免自动显示

html_anim  # 在最后显示动画

In [ ]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 100  # 单位是 MB，设为 100MB 足够


# 1. 系统维度
N = 5               # 腔体最大光子数
wc = 1.0             # 腔体频率
wq = 1.0             # Qubit 频率
g = 0.2              # 耦合常数

# 2. 系统算符 (tensor组合)
a  = tensor(destroy(N), qeye(2))         # 腔体湮灭算符
adag = a.dag()
sz = tensor(qeye(N), sigmaz())           # Qubit Z
sp = tensor(qeye(N), sigmap())
sm = tensor(qeye(N), sigmam())

# 3. Hamiltonian: Jaynes-Cummings
H = wc * adag * a + 0.5 * wq * sz + g * (adag * sm + a * sp)

# 4. 初始态：coherent光场 + 激发态qubit
psi0 = tensor(coherent(N, alpha=2.0), basis(2, 1))  # |α⟩ ⊗ |1⟩

# 5. 时间
tlist = np.linspace(0, 2 * np.pi, 30)

# 6. 无耗散演化
result = mesolve(H, psi0, tlist)

# 7. 动画展示
fig, anim = anim_matrix_histogram(result.states, limits=[-1, 1], colorbar=True)
html_anim = HTML(anim.to_jshtml())  # 保留引用
plt.close(fig)                      # 只关闭刚刚创建的fig，避免自动显示

html_anim  # 在最后显示动画




In [ ]:

# 构造 |00⟩⟨00| 态
ket0 = basis(5, 0)
rho = tensor(ket0, ket0) * tensor(ket0, ket0).dag()

# 获取实部、虚部、模值
rho_real = rho.full().real
rho_imag = rho.full().imag
rho_abs  = np.abs(rho.full())

# 创建子图
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# 实部热力图
im0 = axes[0].imshow(rho_real, cmap='coolwarm')
axes[0].set_title("Real part")
fig.colorbar(im0, ax=axes[0])

# 虚部热力图
im1 = axes[1].imshow(rho_imag, cmap='coolwarm')
axes[1].set_title("Imag part")
fig.colorbar(im1, ax=axes[1])

# 模值热力图
im2 = axes[2].imshow(rho_abs, cmap='viridis')
axes[2].set_title("|rho|")
fig.colorbar(im2, ax=axes[2])

# 设置统一格式
for ax in axes:
    ax.set_xlabel("Column index")
    ax.set_ylabel("Row index")

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.animation as animation
from qutip import basis, mesolve, destroy, qeye, tensor

# 参数
N = 5
a = destroy(N)
H = a.dag() * a
psi0 = basis(N, 2)
tlist = np.linspace(0, 10, 100)

# 演化求解
result = mesolve(H, psi0, tlist)

# 提取每个状态对应的密度矩阵实部
rho_seq = [state.proj().full().real for state in result.states]

# 动画绘图设置
fig, ax = plt.subplots()
im = ax.imshow(rho_seq[0], cmap='coolwarm', vmin=-1, vmax=1)
cbar = plt.colorbar(im)
ax.set_title("Re[ρ(t)]")
ax.set_xlabel("Col index")
ax.set_ylabel("Row index")

# 动画更新函数
def update(frame):
    im.set_array(rho_seq[frame])
    ax.set_title(f"Re[ρ(t)] at t = {tlist[frame]:.2f}")
    return [im]

# 动画对象
ani = animation.FuncAnimation(fig, update, frames=len(rho_seq), interval=100, blit=False)

plt.show()

HTML(ani.to_jshtml())

In [ ]:
# 参数
N = 5  # Hilbert space dimension
alpha = 1.5
a = destroy(N)

# Hamiltonian: H = a†a (number operator)
H = a.dag() * a

# initial state: coherent state
psi0 = coherent(N, alpha)
rho0 = ket2dm(psi0)


# time evolution
tlist = np.linspace(0, 10, 100)

# 演化求解
result = mesolve(H, psi0, tlist)

# 提取每个状态对应的密度矩阵实部
rho_seq = [state.proj().full().real for state in result.states]

# 动画绘图设置
fig, ax = plt.subplots()
im = ax.imshow(rho_seq[0], cmap='coolwarm', vmin=-1, vmax=1)
cbar = plt.colorbar(im)
ax.set_title("Re[ρ(t)]")
ax.set_xlabel("Col index")
ax.set_ylabel("Row index")

# 动画更新函数
def update(frame):
    im.set_array(rho_seq[frame])
    ax.set_title(f"Re[ρ(t)] at t = {tlist[frame]:.2f}")
    return [im]

# 动画对象
ani = animation.FuncAnimation(fig, update, frames=len(rho_seq), interval=100, blit=False)

plt.show()

HTML(ani.to_jshtml())

In [ ]:
# thermal state
n = 1  # average number of thermal photons
thermal_dm(8, n)

In [ ]:
# thermal state
n = 2  # average number of thermal photons
thermal_dm(8, n)

热态（**thermal state**）是量子系统与 **热浴（heat bath）** 处于 **热平衡（thermal equilibrium）** 时的混态（mixed state），非常常见于光学腔、热环境耦合、开放系统等背景。

---

#### 什么是 thermal state?

它是按照玻尔兹曼分布，对 **Fock 态 $|n\rangle$** 加权得到的 **混态密度矩阵**：

$$
\rho_{\text{thermal}} = \sum_{n=0}^\infty \frac{\bar{n}^n}{(1+\bar{n})^{n+1}} |n\rangle \langle n|
$$

* $\bar{n}$：平均热光子数（由温度和频率决定）
* 是对 **多个 Fock 态的概率混合**
* 完全对角化，**没有相干项**

---
#### thermal photon（热光子）是什么？
**thermal photon（热光子）** 是指处于 **热辐射平衡态** 下的光子，来源于**热源（如黑体辐射）**，其统计行为遵循 **玻色分布**，即：

$$
\bar{n} = \frac{1}{e^{\hbar \omega / k_B T} - 1}
$$

以及：
>thermal photon ≈ 来自“热环境”的光子，是**非相干光源**，用于模拟真实系统噪声、耗散、热化等现象。
---

##### 理解热光子

| 特性               | 描述                        |           
| ---------------- | ------------------------- |
| 📦 来源            | 热源（如热腔、热背景辐射、黑体）          |           
| 📈 分布            | 遵循玻色子统计（Bose-Einstein 分布） |           
| 📉 平均数 $\bar{n}$ | 随温度 $T$ 增大而增加             |           
| 🎲 行为            | 是 **无相干性的随机波包组合**，无法干涉    |           
| 🧊 低温极限          | $\bar{n} \to 0$，趋于真空态 (  $\|0\rangle$) |

---

##### 举例直观理解

* 真空态 $|0\rangle$ → $\bar{n} = 0$，零热光子
* 有温度时，可能在 $|0⟩$, $|1⟩$, $|2⟩$... 出现，每个概率符合玻色分布
* 随 $\bar{n}$ 增加，Fock 态分布更宽广

---

##### QuTiP 中生成热光子态


from qutip import thermal_dm
thermal_dm(N=10, n=1.5)  # 10维希尔伯特空间，平均1.5个热光子


结果是对角密度矩阵：

$$
\rho = \sum_n p_n |n\rangle\langle n|
$$




---

#### 为什么常不用它来构造量子信息态？

| 特性           | 描述                             |
| ------------ | ------------------------------ |
| ✅ 简单对角矩阵     | 没有相干性，没有纠缠                     |
| ❌ 不具备量子相干性   | 无法用于构建量子算法、叠加、干涉等关键行为          |
| ❌ 不具有纠缠结构    | 不能作为量子信息资源（如 Bell 态）           |
| ✅ 可用于耗散/噪声环境 | 热态常作为 Lindblad 主方程中的稳态、耗散通道的目标 |

---

#### 用途总结

| 场景             | 是否适合用 thermal state |
| -------------- | ------------------- |
| 理想纯态模拟         | ❌                   |
| 相干叠加与纠缠研究      | ❌                   |
| 模拟耗散/热环境/退相干   | ✅                   |
| Lindblad 动力学初态 | ✅                   |

### Operators

#### Qubit (two-level system) operators

In [ ]:
# Pauli sigma x
sigmax()

In [ ]:
# Pauli sigma y
sigmay()

In [ ]:
# Pauli sigma z
sigmaz()

#### Harmonic oscillator operators

In [ ]:
#  annihilation operator

destroy(N=8)  # N = number of fock states included in the Hilbert space

#### 数学形式

$$
\hat{a} = \sum_{n=1}^{\infty} \sqrt{n} \, |n-1\rangle \langle n|
$$

含义：

* $\hat{a} |n\rangle = \sqrt{n} \, |n-1\rangle$
* 把 Fock 态 $|n\rangle$ **降低一个量子数**（光子数/激发数）
* 是非厄米的：$\hat{a}^\dagger$ 是 **产生算符（creation operator）**

---

#### 图中矩阵含义

用 `destroy(N=8)` 得到了：

* 一个 $8 \times 8$ 矩阵
* 非零元出现在 **主对角线上方一条线**，即：上三角矩阵结构（superdiagonal）
* 第 $n,n+1$ 元素 = $\sqrt{n+1}$

例子：

$$
\hat{a} =
\begin{pmatrix}
0 & 1 & 0 & 0 & \cdots \\
0 & 0 & \sqrt{2} & 0 & \cdots \\
0 & 0 & 0 & \sqrt{3} & \cdots \\
\vdots & \vdots & \vdots & \ddots & \ddots \\
\end{pmatrix}
$$

---

#### 用途

* 构造哈密顿量（如 $\hat{a}^\dagger \hat{a}$、Jaynes-Cummings）
* 构造 coherent 态：$|\alpha\rangle = e^{-|\alpha|^2/2} \sum \frac{\alpha^n}{\sqrt{n!}} |n\rangle$
* Lindblad 耗散通道：$L = \sqrt{\kappa} \, \hat{a}$

---

In [ ]:
# creation operator

create(N=8)  # equivalent to destroy(5).dag()

**creation operator（产生算符）** $\hat{a}^\dagger$，是 **湮灭算符（$\hat{a}$）的厄米共轭**，用于“增加”一个量子激发（比如光子、粒子等）。

---

#### 数学表达式：

$$
\hat{a}^\dagger = \sum_{n=0}^{\infty} \sqrt{n+1} \, |n+1\rangle \langle n|
$$

---

#### 物理意义：

* $\hat{a}^\dagger |n\rangle = \sqrt{n+1} |n+1\rangle$：
  将状态 $|n\rangle$ **提升为** $|n+1\rangle$

* 与湮灭算符相反，它是用来“**加粒子**”的算符

---

#### 图中结构说明：

```python
create(N=8)
```

输出的是 $8 \times 8$ 矩阵，其中：

* 非零元在主对角下方一条线即：下三角矩阵结构（subdiagonal）
* 第 $(n+1, n)$ 元素为 $\sqrt{n+1}$

比如：

$$
\begin{bmatrix}
0 & 0 & 0 & \cdots \\
1 & 0 & 0 & \cdots \\
0 & \sqrt{2} & 0 & \cdots \\
0 & 0 & \sqrt{3} & \cdots \\
\vdots & \vdots & \vdots & \ddots
\end{bmatrix}
$$

---

#### 应用场景：

* 构造哈密顿量：$\hat{a}^\dagger \hat{a}$、$\hat{a}^\dagger + \hat{a}$
* 构造 coherent 态
* 与湮灭算符组合形成拉达算符（ladder operators）

---

In [ ]:
# the position operator is easily constructed from the annihilation operator
a = destroy(8)

x = a + a.dag()

x

**这个 position operator $\hat{x} = \hat{a} + \hat{a}^\dagger$**，其实是**作用在谐振子态空间中的“无量纲位置算符”**。

---

#### 它表示的是“谁的”位置？

>是**量子简谐振子系统中粒子的“位置”**。
* 例如：

    * 光场模式中的光子（腔 QED 模型）
    * 磁场中带电粒子的振荡模式
    * 量子力学中一维势阱的质点位置
    * 也可以是冷原子/机械共振器的运动自由度等

---

#### 为什么写成 $\hat{x} = \hat{a} + \hat{a}^\dagger$？

在量子谐振子的自然单位（$\hbar = m = \omega = 1$）下，位置与动量算符可以通过产生/湮灭算符构造：

$$
\hat{x} = \frac{1}{\sqrt{2}}(\hat{a} + \hat{a}^\dagger), \quad \hat{p} = \frac{i}{\sqrt{2}}(\hat{a}^\dagger - \hat{a})
$$

这段代码其实少了系数 $\frac{1}{\sqrt{2}}$，但它仍表示的是“位置算符结构”，只差个缩放因子。

---

#### 它作用在谁身上？

它作用在 Fock 空间中的基态 $|n\rangle$，即：

* $|0\rangle, |1\rangle, \dots, |N-1\rangle$
* 这里用了 `destroy(8)`，所以是作用在维度为 8 的 Fock 空间。

---

#### 它的物理意义是？

这个算符用来：

* 模拟位置空间观测
* 构造 Hamiltonian 中的势能项，如 $H = \frac{p^2}{2m} + \frac{1}{2} m\omega^2 x^2$
* 做量子态的 Wigner 分布可视化（phase-space）
* 用于 cavity-QED/optomechanics/coherent state 等的含时演化分析

---


#### 要模拟“真实空间中的位置观测”，记得加上因子：

In [ ]:
x = (a + a.dag()) / np.sqrt(2)
x

### 构造动量算符$\hat{p}$和哈密顿量$\hat{H} = \frac{1}{2}(\hat{x}^2 + \hat{p}^2)$

---

#### 任务目标

**构建一维量子简谐振子的哈密顿量：**

$$
\hat{H} = \frac{1}{2}(\hat{x}^2 + \hat{p}^2)
$$

其中：

* $\hat{x} = \frac{1}{\sqrt{2}} (a + a^\dagger)$
* $\hat{p} = \frac{i}{\sqrt{2}} (a^\dagger - a)$

我们用 QuTiP 实现该构造并验证其本征态为 Fock 态，能量本征值为 $E_n = n + \frac{1}{2}$。

---

#### 代码规划（伪代码）

1. 定义 Fock 空间维度 $N$
2. 构建湮灭算符 $\hat{a}$，产生算符 $\hat{a}^\dagger$
3. 构建 $\hat{x}, \hat{p}$
4. 构建哈密顿量 $\hat{H} = \frac{1}{2}(x^2 + p^2)$
5. 求其本征值/本征态用于验证
---


In [ ]:
import numpy as np
from qutip import destroy, qeye

# 1. Hilbert space dimension
N = 20

# 2. Create annihilation operator a
a = destroy(N)

# 3. Construct position and momentum operators
x = (a + a.dag()) / np.sqrt(2)
p = 1j * (a.dag() - a) / np.sqrt(2)

# 4. Hamiltonian: H = 0.5 * (x^2 + p^2)
H = 0.5 * (x @ x + p @ p)

# 5. Diagonalize the Hamiltonian
eigvals, eigvecs = H.eigenstates()

# 6. Print first few energy levels
for n, val in enumerate(eigvals[:8]):
    print(f"n={n}, E = {val:.5f}")

>这就验证了简谐振子的本征能量是 $E_n = n + \frac{1}{2}$，说明构造是正确的！

**可视化简谐振子前几个能量本征态对应的密度矩阵**。

---

#### 目标

用 `qutip.matrix_histogram()` 展示：

* 第 0 态（基态）密度矩阵 $\rho_0 = |\psi_0\rangle\langle\psi_0|$
* 第 1 态密度矩阵 $\rho_1 = |\psi_1\rangle\langle\psi_1|$
* 第 2 态密度矩阵 $\rho_2 = |\psi_2\rangle\langle\psi_2|$

---

#### 伪代码结构

1. 构造哈密顿量 $H$
2. 得到本征态列表 `eigvecs`
3. 取前几个态生成密度矩阵
4. `matrix_histogram()` 绘图，简化坐标轴



In [ ]:
# file: oscillator_eigenstates_plot.py

import numpy as np
import matplotlib.pyplot as plt
from qutip import destroy, matrix_histogram
from qutip import Qobj

# 设置维度
N = 20
a = destroy(N)

# 构造 x 和 p
x = (a + a.dag()) / np.sqrt(2)
p = 1j * (a.dag() - a) / np.sqrt(2)

# 构造哈密顿量 H = 0.5(x^2 + p^2)
H = 0.5 * (x @ x + p @ p)

# 求解本征态
eigvals, eigvecs = H.eigenstates()

# 可视化前几个能量态的密度矩阵
for n in range(3):  # 0,1,2态
    psi = eigvecs[n]
    rho = Qobj(psi) * Qobj(psi).dag()  # 生成密度矩阵

    fig = plt.figure(figsize=(6, 5))
    ax = fig.add_subplot(111, projection='3d')
    matrix_histogram(rho, fig=fig, ax=ax)

    # 设置标签简洁化
    dim = rho.shape[0]
    ticks = np.arange(0, dim, 4)  # 每隔4个 tick
    labels = [f"|{i}⟩" for i in ticks]  

    ax.set_xticks(ticks)
    ax.set_xticklabels(labels, rotation=45, fontsize=10)
    ax.set_yticks(ticks)
    ax.set_yticklabels(labels, rotation=45, fontsize=10)
    
    ax.set_title(f"$\\rho = |\\psi_{n}\\rangle\\langle\\psi_{n}|$, E={eigvals[n]:.2f}")
    plt.show()


#### Using `Qobj` instances we can check some well known commutation relations:

In [ ]:
def commutator(op1, op2):
    return op1 * op2 - op2 * op1

$[a, a^1] = 1$

In [ ]:
a = destroy(5)

commutator(a, a.dag())

这里在验证两个矩阵（即量子算符）是否满足**对易关系（commutation relation）**。

---

#### 什么是对易关系？

对两个算符 $A$ 和 $B$，\*\*对易子（commutator）\*\*定义为：

$$
[A, B] = AB - BA
$$

* 如果 $[A, B] = 0$，称 $A$ 和 $B$**对易**（commute）
* 如果 $[A, B] \neq 0$，则称它们**不对易**（do not commute）

---

#### 图中验证的是谁和谁的对易？

```python
a = destroy(5)          # 湮灭算符 a
a_dag = a.dag()         # 产生算符 a†
commutator(a, a_dag)    # [a, a†] = a·a† - a†·a
```

理论上，它们满足：

$$
[a, a^\dagger] = I
$$

即：**湮灭与产生算符的对易子为单位算符**

---

#### 那图中结果为什么有 `-4`？

因为这里是有限维（维度为 5）空间中的近似计算：

```plaintext
Quantum object: shape=(5, 5)
```

而理想的对易关系是对**无穷维希尔伯特空间**成立的。维数太小（比如 5）会导致边界效应（truncation error），所以最后一行最后一列出现了 `-4`。

---

#### 解决办法：升高维数，比如：

```python
a = destroy(50)
commutator(a, a.dag())  # 最后应该越来越接近单位矩阵
```

---


**Ops...** The result is not identity! Why? Because we have truncated the Hilbert space. But that's OK *as long as the highest Fock state isn't involved in the dynamics in our truncated Hilbert space.* If it is, the approximation that the truncation introduces might be a problem.

$[x,p] = i$

In [ ]:
x = (a + a.dag()) / np.sqrt(2)
p = -1j * (a - a.dag()) / np.sqrt(2)

In [ ]:
commutator(x, p)

Same issue with the truncated Hilbert space, but otherwise OK.

Let's try some Pauli spin inequalities

$[\sigma_x, \sigma_y] = 2i \sigma_z$

In [ ]:
commutator(sigmax(), sigmay()) - 2j * sigmaz()

$-i \sigma_x \sigma_y \sigma_z = \mathbf{1}$

In [ ]:
-1j * sigmax() * sigmay() * sigmaz()

$\sigma_x^2 = \sigma_y^2 = \sigma_z^2 = \mathbf{1}$

In [ ]:
sigmax() ** 2 == sigmay() ** 2 == sigmaz() ** 2 == qeye(2)
# qeye(2)是一个 2×2 的单位矩阵,
# 用于量子计算中构建 Hilbert 空间的单位算符（identity operator）。

## Composite systems

In most cases we are interested in **coupled quantum systems**, for example *coupled qubits*, *a qubit coupled to a cavity (oscillator mode)*, etc.

To define states and operators for such systems in QuTiP, we use the `tensor` function to create `Qobj` instances for the composite system.

在量子计算与量子力学中，**Composite system（复合系统）** 是指由多个**子系统（subsystems）**组合而成的整体系统。
每个子系统都有自己的希尔伯特空间，整个系统的空间是它们的**张量积**。

---

#### 举例说明：

如果有两个量子比特：

* 第一个量子比特的状态空间是：                                            

  $$
  \mathcal{H}_1 = \text{span}\{|0\rangle, |1\rangle\}
  $$
* 第二个量子比特的状态空间是：                                            

  $$
  \mathcal{H}_2 = \text{span}\{|0\rangle, |1\rangle\}
  $$

那么**复合系统**的总空间就是两个希尔伯特空间的**张量积**：

$$
\mathcal{H}_{\text{total}} = \mathcal{H}_1 \otimes \mathcal{H}_2
= \text{span}\{ |00\rangle, |01\rangle, |10\rangle, |11\rangle \}
$$

---

#### 总结：

| 子系统数      | 状态空间大小  | 示例            |         
| --------- | ------- | ------------- |
| 1 个 qubit | 2       | ( $\|0\rangle$,  $\|1\rangle)$  |     
| 2 个 qubit | 2×2 = 4 | ( $\|00\rangle, \|01\rangle, \|10\rangle, \|11\rangle)$ |
| n 个 qubit | $2^n$   | $2^n$ 维希尔伯特空间 |         

---

#### 在 QuTiP 中的构造：

In [ ]:
from qutip import tensor, basis

psi1 = basis(2, 0)  # |0>
psi2 = basis(2, 1)  # |1>
psi_total = tensor(psi1, psi2)  # |0⟩⊗|1⟩ = |01⟩

psi_total

For example, consider a system composed of two qubits. If we want to create a Pauli $\sigma_z$ operator that acts on the first qubit and leaves the second qubit unaffected (i.e., the operator $\sigma_z \otimes \mathbf{1}$), we would do:

In [ ]:
sz1 = tensor(sigmaz(), qeye(2))

sz1

We can easily verify that this two-qubit operator does indeed have the desired properties:

In [ ]:
N = 2
psi1 = tensor(basis(N, 1), basis(N, 0))  # excited first qubit
psi2 = tensor(basis(N, 0), basis(N, 1))  # excited second qubit


In [ ]:
# this should not be true,
# because sz1 should flip the sign of the excited state of psi1
sz1 * psi1 == psi1

>sz1 does affect the excited first qubit

In [ ]:
# this should be true, because sz1 should leave psi2 unaffected
sz1 * psi2 == psi2

Above we used the `qeye(N)` function, which generates the identity operator with `N` quantum states. If we want to do the same thing for the second qubit we can do:

In [ ]:
sz2 = tensor(qeye(2), sigmaz())

sz2

Note the order of the argument to the `tensor` function, and the correspondingly different matrix representation of the two operators `sz1` and `sz2`.

Using the same method we can create coupling terms of the form $\sigma_x \otimes \sigma_x$:

In [ ]:
tensor(sigmax(), sigmax())

Now we are ready to create a `Qobj` representation of a coupled two-qubit Hamiltonian: $H = \epsilon_1 \sigma_z^{(1)} + \epsilon_2 \sigma_z^{(2)} + g \sigma_x^{(1)}\sigma_x^{(2)}$

In [ ]:
epsilon = [1.0, 1.0]
g = 0.1

sz1 = tensor(sigmaz(), qeye(2))
sz2 = tensor(qeye(2), sigmaz())

H = epsilon[0] * sz1 + epsilon[1] * sz2 + g * tensor(sigmax(), sigmax())

H

### 逐项拆解这个 Hamiltonian 构建：

---

#### 总哈密顿量结构

你构建的是一个 **两个量子比特耦合系统的哈密顿量**：

$$
H = \epsilon_1 \sigma_z^{(1)} + \epsilon_2 \sigma_z^{(2)} + g \, \sigma_x^{(1)} \sigma_x^{(2)}
$$

这是一个典型的 **横场 Ising 模型（Transverse Field Ising Model）**。

---

#### 每一项含义解释

#### ① `sz1 = tensor(sigmaz(), qeye(2))`

* 含义：第一个 qubit 上的 $\sigma_z$ 算符，第二个 qubit 不变（单位）
* 数学形式：$\sigma_z \otimes I$
* 用途：$\epsilon_1 \sigma_z^{(1)}$

#### ② `sz2 = tensor(qeye(2), sigmaz())`

* 含义：第二个 qubit 上的 $\sigma_z$，第一个不变
* 数学形式：$I \otimes \sigma_z$
* 用途：$\epsilon_2 \sigma_z^{(2)}$

#### ③ `tensor(sigmax(), sigmax())`

* 含义：两个 qubit 上都施加 $\sigma_x$
* 数学形式：$\sigma_x \otimes \sigma_x$
* 用途：表示两个 qubit 的耦合（相互作用）

---

#### 整体构建代码

```python
epsilon = [1.0, 1.0]     # 每个 qubit 的能级偏置
g = 0.1                  # 两个qubit的耦合强度

sz1 = tensor(sigmaz(), qeye(2))   # 第一个 qubit 的 sz
sz2 = tensor(qeye(2), sigmaz())   # 第二个 qubit 的 sz

H = epsilon[0]*sz1 + epsilon[1]*sz2 + g * tensor(sigmax(), sigmax())
```

最终 `H` 是 4x4 Hermitian 哈密顿量矩阵。

---

#### 总结

| 项        | 数学形式                                  | 意义                |
| -------- | ------------------------------------- | ----------------- |
| `ε₁ sz₁` | $\epsilon_1 \sigma_z^{(1)}$       | 第1个量子比特的偏置能量      |
| `ε₂ sz₂` | $\epsilon_2 \sigma_z^{(2)}$       | 第2个量子比特的偏置能量      |
| `g XX` | $g \sigma_x^{(1)} \sigma_x^{(2)}$ | 两个量子比特之间的相互作用（耦合） |




---

### `sz1` 和 `sz2` 的物理作用

>### 总结

| 名称    | 数学作用                              | 物理含义              |                 
| ----- | --------------------------------- | ----------------- | 
| `sz1` | $\sigma_z \otimes I$           | 改变第一个 qubit 的能量偏置 |               
| `sz2` | $I \otimes \sigma_z$           | 改变第二个 qubit 的能量偏置 |                 
| 偏置能量  | $\epsilon_i \sigma_z^{(i)}$ 项 | 控制 $\|0\rangle$ 和 $\|1\rangle$ 的能量差异 |

---

先复习两个构造：

```python
sz1 = tensor(sigmaz(), qeye(2))  # σ_z 作用在第一个qubit上
sz2 = tensor(qeye(2), sigmaz())  # σ_z 作用在第二个qubit上
```

它们分别表示：

* `sz1`: 对 **第一个 qubit** 施加 Pauli Z 算符，**第二个不变**
* `sz2`: 对 **第二个 qubit** 施加 Pauli Z 算符，**第一个不变**

---

#### Pauli-Z 算符作用回顾：

$$
\sigma_z |0\rangle = |0\rangle,\quad \sigma_z |1\rangle = -|1\rangle
$$

即：

* 保持基态 $|0\rangle$
* 翻转激发态 $|1\rangle$ 的符号（sign flip）

---

#### 所以作用总结如下：

| 算符    | 作用对象      | 效果              |                   
| ----- | --------- | --------------- | 
| `sz1` | 第一个 qubit | 仅第一个 qubit 的 $\|1\rangle$ 被乘以 -1 |
| `sz2` | 第二个 qubit | 仅第二个 qubit 的 $\|1\rangle$ 被乘以 -1 |

---

### “偏置能量” 是什么？

“偏置能量”（bias energy）是量子比特在 $\sigma_z$ 方向上的外加场/能量差：

在哈密顿量项中：

$$
H = \epsilon_1 \sigma_z^{(1)} + \epsilon_2 \sigma_z^{(2)} + \cdots
$$

其中：

* $\epsilon_1$ 和 $\epsilon_2$ 决定每个 qubit 的 **本征能级差（能量不对称）**
* 越大代表 $|0\rangle$ 和 $|1\rangle$ 的能量差越大（类似施加静态磁场）

---

#### 举例说明（对第一个 qubit）：

若只考虑 $\epsilon_1 \sigma_z^{(1)}$：

$$
\sigma_z^{(1)} |00\rangle = |00\rangle,\quad
\sigma_z^{(1)} |10\rangle = -|10\rangle
$$

那么：

* $|00\rangle$ 变为 $\epsilon_1 |00\rangle$
* $|10\rangle$ 变为 $-\epsilon_1 |10\rangle$

说明：**这就是能量偏置 —— 量子比特的两个状态不再等能了**

---





### 为什么要施加 Pauli-Z（$σ_z$）？

>**施加 $σ_z$ 项就是为了人为地引入能级不对称**，即所谓的 **偏置能量**，从而：

* 控制 qubit 的演化（调节其“振荡频率”）
* 实现量子门操作（比如 $Z$ 门或 $R_z(\theta)$）
* 区分不同 qubit，防止它们互相干扰（频率分离）

---

#### $σ_z$ 项对应的哈密顿量含义：

哈密顿量项 $\epsilon \cdot \sigma_z$ 代表一个 qubit 在外加 **静态磁场/电场** 中的能量偏置

$$
H = \epsilon \cdot \sigma_z \Rightarrow
\begin{cases}
|0\rangle \rightarrow +\epsilon \text{ energy} \\
|1\rangle \rightarrow -\epsilon \text{ energy}
\end{cases}
$$

这相当于将 $|0\rangle$ 和 $|1\rangle$ 的能量差拉开，使 qubit 有一个确定的能级结构。

---

### 那为啥要引入偏置能量？

偏置能量是**为了控制 qubit**，它是你在量子电路/超导 qubit 实验中**最重要的调控参数之一**。

#### 用途总结如下：

| 用途            | 描述                                      |                
| ------------- | --------------------------------------- | 
| 🧭 控制量子态的相位演化 | 让 qubit 在 $\|0\rangle$ 与 $\|1\rangle$ 间“旋转”，演化有节奏 |
| 📡 实现频率选择性    | 调节不同 qubit 的频率避免互串干扰（比如调 5GHz vs 6GHz）  |                                         |
| 🧱 构建门操作      | 你可以调控 $\epsilon(t)$ 去完成 Z 门 或 更复杂的量子门 |                                        
| 🧪 区分量子态能级    | 没有偏置，$\|0\rangle$ 和 $\|1\rangle$ 是简并的，信息无法编码 |

---

#### 为什么是偏置能量（$σ_z$）而不是其他？

$σ_x$ 或 $σ_y$ 会导致 **翻转、激发跃迁、共振行为**，而 $σ_z$ 只会加上一个 **相对相位/能量差** —— 它是“稳定的、可控的背景场”。

例如：

* `σ_x`：诱导态跃迁（Rabi 振荡、驱动 qubit）
* `σ_z`：能量调控，不改变态，**只让相位积累变化**

这是量子控制中经典的 **分工协作**：

| Pauli项 | 用于控制      | 说明           |                   
| ------ | --------- | ------------ | 
| $σ_z$   | 偏置能量、静态调控 | 调节能级结构，不激发跃迁 |                  
| $σ_x$   | 激发跃迁、门操作  | 促使 $\|0⟩\leftrightarrow \|1⟩$ 转换 |

---

#### 物理实现示意（比如在超导量子比特）

你可以通过：

* 控制磁通 $\Phi$（改变能级间距）$\rightarrow$ 对应 $\epsilon \sigma_z$
* 加微波脉冲（横向激发）$\rightarrow$ 对应 $\sigma_x$、$\sigma_y$ 控制项

---

### 总结

> **Pauli-Z 项是为了给每个 qubit 加上一个静态能量偏置，让系统“可控”又“可区分”。**

| 概念       | 解释                                 |
| -------- | ---------------------------------- |
| $σ_z$     | 设定基态/激发态能级差                        |
| 偏置能量 ε   | 控制 qubit 的频率、相位积累、状态演化             |
| $σ_z$ 的作用 | 保持态不变，但添加相位（控制 Z 方向演化）             |
| 为什么不用别的  | $σ_x/σ_y$ 用于跃迁，$σ_z$ 用于能级结构调控，是互补关系 |



### **量子控制与门操作的核心逻辑**：

---

### 加入偏置项（σ<sub>z</sub>）**就是为了让每个 qubit 独立可控**：

#### 真实场景中的两个 qubit：

假设你有两个 qubit（超导、离子阱或光学）：

* 它们通过某种机制耦合（比如电容、电感、腔模）
* 如果不做处理，它们会自动发生纠缠、串扰

---

#### 加入偏置项后会发生什么：

$$
H = \epsilon_1 \sigma_z^{(1)} + \epsilon_2 \sigma_z^{(2)} + g \sigma_x^{(1)} \sigma_x^{(2)}
$$

* 加入 σ<sub>z</sub> 项后，每个 qubit 都有一个**不同的能级间距**
* 你可以通过 ε₁ ≠ ε₂ 来**“解耦”它们在频域上的耦合**

也就是：

> **频率不同 → 彼此不共振 → 不自动耦合 → 不纠缠**

---

### 然后呢？为什么这么做？

#### 为了做“单比特门操作”

如果你不先“区分开”它们，你就无法选择性地：

* 对 qubit A 施加一个 X 门（π-pulse）
* 而 qubit B 保持不动

---

### 类比理解：调频道 & 精准打击

你可以把每个 qubit 看作是一个收音机的频道：

| Qubit | 偏置后频率   | 用于选择性控制        |
| ----- | ------- | -------------- |
| Q1    | 5.0 GHz | 用 5.0 GHz 脉冲激发 |
| Q2    | 6.5 GHz | 用 6.5 GHz 脉冲激发 |

这样做，你可以：

* 精准控制 Q1 而不影响 Q2
* 做单比特门、调控相位、不引入耦合

---

### 剥离出耦合的 qubit，用于什么？

| 操作目的            | 原因                                                   |
| --------------- | ---------------------------------------------------- |
| 施加单量子门（X, Y, Z） | 只控制一个 qubit                                          |
| 初始化             | 防止纠缠态出现在一开始                                          |
| 编程逻辑门序列         | 保证门序列的可预测性与可控性                                       |
| 编织纠缠态           | 在你想纠缠时，再打开放耦部分 $g \sigma_x^{(1)} \sigma_x^{(2)}$ |

---

### 总结这句话：

> **加上 σ<sub>z</sub> 偏置项 → 人为区分 qubit → 解耦它们 → 可以单独操作**







## 耦合系统的哈密顿量结构 ：

---

#### 总体形式回顾：

$$
H = \epsilon_1 \sigma_z^{(1)} + \epsilon_2 \sigma_z^{(2)} + g \, \sigma_x^{(1)} \sigma_x^{(2)}
$$

---

#### 前两项：**单量子比特能量项**

| 项                               | 物理含义          | 操作对象      | 本质上做了什么        |                      
| ------------------------------- | ------------- | --------- | -------------- | 
| $\epsilon_1 \sigma_z^{(1)}$ | Qubit 1 的偏置能量 | 对 Qubit 1 | 控制其 $\left\|0\right\rangle, \left \|1\right\rangle$ 的能级差 |
| $\epsilon_2 \sigma_z^{(2)}$ | Qubit 2 的偏置能量 | 对 Qubit 2 | 同上，调控能级偏置，分离频率 |                  

也就是：

* 这两个项各自为 qubit 提供 “**Z 方向的能量倾斜**”
* 数学上是 Pauli-Z，物理上就是能级间距（$E_1 - E_0$）
* 你可以选择让两个 qubit：

  * 相同频率（共振）
  * 不同频率（频率分离）

---

#### 第三项：**两个量子比特的耦合项**

$$
g \cdot \sigma_x^{(1)} \sigma_x^{(2)}
$$

| 项                                   | 物理含义          | 数学效果                |
| ----------------------------------- | ------------- | ------------------- |
| $g$                               | 耦合强度（tunable） | 表示能量交换幅度            |
| $\sigma_x^{(1)} \sigma_x^{(2)}$ | X 通道上的能量交换    | 将一个 qubit 的激发传递给另一个 |

这个项允许出现：

* Qubit 1 激发 → Qubit 2 被激发（或反向）
* 产生纠缠 $\left|01\right\rangle \leftrightarrow \left|10\right\rangle$

---

#### 总结一句话：

> **这个哈密顿量中：**
>
> * Z 项提供局部能量结构（偏置）→ 控制每个 qubit 自身
> * X-X 项提供耦合结构 → 控制两个 qubit 之间的相互作用

这种分离 **非常重要**，因为：

* 它让你可以自由切换：

  * **只控 qubit 本身**（关掉耦合）
  * **让 qubit 相互作用**（开启耦合）

---


To create composite systems of different types, all we need to do is to change the operators that we pass to the `tensor` function (which can take an arbitrary number of operator for composite systems with many components).

For example, **the Jaynes-Cumming Hamiltonian** for a qubit-cavity system:

$H = \omega_c a^\dagger a - \frac{1}{2}\omega_a \sigma_z + g (a \sigma_+ + a^\dagger \sigma_-)$

In [ ]:
wc = 1.0  # cavity frequency
wa = 1.0  # qubit/atom frenqency
g = 0.1  # coupling strength

# cavity mode operator
a = tensor(destroy(5), qeye(2))

# qubit/atom operators
sz = tensor(qeye(5), sigmaz())  # sigma-z operator
sm = tensor(qeye(5), destroy(2))  # sigma-minus operator

# the Jaynes-Cumming Hamiltonian
H = wc * a.dag() * a - 0.5 * wa * sz + g * (a * sm.dag() + a.dag() * sm)

H

Note that

$a \sigma_+ = (a \otimes \mathbf{1}) (\mathbf{1} \otimes \sigma_+)$

so the following two are identical:

In [ ]:
a = tensor(destroy(3), qeye(2))
sp = tensor(qeye(3), create(2))

a * sp

In [ ]:
tensor(destroy(3), create(2))

## Unitary dynamics

Unitary evolution of a quantum system in QuTiP can be calculated with the `mesolve` function.

`mesolve` is short for **Master-eqaution solve (for dissipative dynamics)**, but *if no collapse operators (which describe the dissipation) are given to the solve* it falls back on **the unitary evolution of the Schrodinger (for initial states in state vector for) or the von Neuman equation (for initial states in density matrix form)**.

The evolution solvers in QuTiP returns a class of type `Odedata`, which contains the solution to the problem posed to the evolution solver.

For example, considor a qubit with Hamiltonian $H = \sigma_x$ and initial state $\left|1\right>$ (in the sigma-z basis): Its evolution can be calculated as follows:

In [ ]:
# Hamiltonian
H = sigmax()

# initial state
psi0 = basis(2, 0)

# list of times for which the solver should store the state vector
tlist = np.linspace(0, 10, 100)

result = mesolve(H, psi0, tlist, [], [])

#### Warning 的含义
警告提示：

>在未来的 QuTiP 5.3 版本中，e_ops 只能通过关键字传入。<br>
>即，result = mesolve(H, psi0, tlist, [], e_ops=e_ops)


`e_ops`（expectation operators）参数用于告诉 `mesolve`（或其他 QuTiP 动力学求解器）：

> 在每个时间步上，**你想观察哪个算符的期望值**。

---

#### 解释：

```python
result = mesolve(H, psi0, tlist, [], e_ops)
```

* `e_ops`: 列表，包含你希望监控的算符
* 结果中：`result.expect[i]` 就是第 `i` 个算符在所有时间点上的期望值

---

#### 示例：加入测量期望值
想测量 $\langle \sigma_z \rangle$ 的随时间演化：

In [ ]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt

# Hamiltonian
H = tensor(sigmax(), qeye(2))  # 显式作用于第一个量子比特

# 初始态（2-qubit 系统：第一个比特为 |0>，第二个为 |0>）
psi0 = tensor(basis(2, 0), basis(2, 0))

# 时间列表
tlist = np.linspace(0, 10, 100)

# 期望值观测（作用在第一个 qubit 上的 sigmaz）
e_ops = [tensor(sigmaz(), qeye(2))]

# 演化
result = mesolve(H, psi0, tlist, [], e_ops)

# 可视化
plt.plot(tlist, result.expect[0])
plt.xlabel('Time')
plt.ylabel('<σz>')
plt.show()


> 这是**期望值 $\langle \sigma_z \rangle$ 随时间的演化图像**。

---

### 为什么纵轴可以是负的？

期望值（expectation value）表示的是**对一个状态测量时的平均结果**，它可以是正、负或零，取决于你测的算符和系统当前的状态。

---

#### 举个例子：测量 $\sigma_z$ 的期望值

$$
\sigma_z =
\begin{pmatrix}
1 & 0 \\
0 & -1
\end{pmatrix}
$$

如果系统处于基态 $|0\rangle$：

$$
\langle \sigma_z \rangle = \langle 0 | \sigma_z | 0 \rangle = 1
$$

如果系统处于激发态 $|1\rangle$：

$$
\langle \sigma_z \rangle = \langle 1 | \sigma_z | 1 \rangle = -1
$$

---

#### 所以你的图是什么意思？

你的系统初始是 $|0\rangle$，
哈密顿量是 $H = \sigma_x$，这个会导致 qubit 在 $|0\rangle$ 和 $|1\rangle$ 之间震荡（Rabi oscillation）。

所以你看到的：

* $\langle \sigma_z \rangle$ 从 1 降到 -1
* 再回到 1，周期性震荡
* 对应 qubit 在 ${|0\rangle, |1\rangle}$ 之间反复翻转

---

#### 小结

* **纵轴是测量 $\sigma_z$ 的平均值**
* **负值表示“更偏向 $|1\rangle$”的态**
* 整体是量子态演化下的**Bloch 球 Z 分量**

---


用 `matplotlib` 自己的 `3D` 绘图 + `animation` 模块来可视化布洛赫球上的运动轨迹。下面是：

---

#### 用 `matplotlib` 自定义布洛赫球 3D 动画

##### 功能：

* 模拟单量子比特随时间演化的状态
* 使用 `matplotlib` 的 `Axes3D` + `FuncAnimation`
* 动画展示布洛赫球轨迹

---

#### Step-by-step 计划（伪代码）：

1. 设置 Hamiltonian（比如 `sigmax`）
2. 初始态设为基态 `|0⟩`
3. 定义时间序列 `tlist`
4. 用 `mesolve` 计算每个时间步的 `state`
5. 对每个 `state` 计算布洛赫矢量 (x, y, z)
6. 用 `matplotlib`:

   * 绘制布洛赫球
   * `scatter` 当前状态点
   * `line` 绘制路径轨迹
   * 用 `FuncAnimation` 更新帧

---

#### 运行效果

你会看到红点在布洛赫球表面转动，蓝线留下轨迹，表现出量子比特随 Hamiltonian 演化的状态。

---

#### 注意事项

* 不依赖 `qutip.plotting.Bloch`，兼容任何版本
* 自定义球面更灵活，可手动设置颜色/透明度等
* `mesolve` 的结果必须是纯态才能转布洛赫向量

---


In [ ]:
# file: bloch_animation_matplotlib.py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

from qutip import basis, sigmax, sigmay, sigmaz, mesolve

# 1. Hamiltonian: H = σx
H = sigmax() + sigmay() * sigmaz()  # directly affect the trajectory of the point movement

# 2. Initial state |0>
psi0 = basis(2, 0)

# 3. Time evolution
tlist = np.linspace(0, 10, 100)
result = mesolve(H, psi0, tlist, [])

# 4. Extract Bloch vectors
bloch_vectors = np.array([
    [
        np.real((state.dag() * sigmax() * state)),
        np.real((state.dag() * sigmay() * state)),
        np.real((state.dag() * sigmaz() * state))
    ]
    for state in result.states
])


# 5. Setup 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Bloch Sphere Animation')

# Draw Bloch sphere wireframe
u = np.linspace(0, 2 * np.pi, 50)
v = np.linspace(0, np.pi, 50)
x_sphere = np.outer(np.cos(u), np.sin(v))
y_sphere = np.outer(np.sin(u), np.sin(v))
z_sphere = np.outer(np.ones_like(u), np.cos(v))
ax.plot_wireframe(x_sphere, y_sphere, z_sphere, color='lightgray', alpha=0.3)

# Init animation objects
point, = ax.plot([], [], [], 'ro')
path, = ax.plot([], [], [], 'b-', lw=2)

# 6. Animation function
def update(i):
    path.set_data(bloch_vectors[:i, 0], bloch_vectors[:i, 1])
    path.set_3d_properties(bloch_vectors[:i, 2])
    point.set_data(bloch_vectors[i:i+1, 0], bloch_vectors[i:i+1, 1])
    point.set_3d_properties(bloch_vectors[i:i+1, 2])
    return point, path

ani = FuncAnimation(fig, update, frames=len(tlist), interval=100, blit=True)
plt.show()

ani


In [ ]:
result

这是 `qutip.mesolve` 的执行结果对象 `<Result>`，它提供了关于求解器（solver）运行的信息。下面逐项来解读：

---

#### `result` 输出解读

```text
<Result
 Solver: sesolve
```

* 使用的求解器是 `sesolve`，即 Schrödinger equation solver（用于纯态 time evolution）。

---

```text
 Solver stats:
   method: 'scipy zvode adams'
```

* 数值积分方法是 SciPy 中的 ZVODE 的 `adams` 方法，适用于非刚性问题。

---

```text
   init time: 0.000166...
   preparation time: 0.00030...
   run time: 0.00189...
```

* 初始化、准备和运行所花的时间，单位是秒（s），这些是内部性能统计信息。

---

```text
   solver: 'Schrodinger Evolution'
```

* 指明使用的是求解薛定谔方程的 solver（即用于纯态，不是密度矩阵的 Lindblad 方程）。

---

```text
Time interval: [0.0, 10.0] (100 steps)
```

* 模拟时间从 `0.0` 到 `10.0`，一共划分为 `100` 个时间点（由 `tlist = np.linspace(0, 10, 100)` 给出）。

---

```text
Number of e_ops: 0
```

* 你没有传入 `e_ops`，即没有指定要计算哪些算符的期望值（expectation values），所以是 0。

---

```text
States saved.
```

* 所有时间点下的状态（state）已被保存，可以通过 `result.states` 访问。

---


In [ ]:
# result.states         # 所有时间点下的波函数（state vector）

In [ ]:
# result.times          # 时间列表（tlist）

In [ ]:
# result.states[0]  # t=0 时刻的态

In [ ]:
# from qutip import sigmax, sigmay, sigmaz
# expect_vals_x = expect(sigmax(), result.states)
# expect_vals_y = expect(sigmay(), result.states)
# expect_vals_z = expect(sigmaz(), result.states)
# plt.plot(result.times, expect_vals_x)
# plt.plot(result.times, expect_vals_y)
# plt.plot(result.times, expect_vals_z)

The `result` object contains a list of the wavefunctions at the times requested with the `tlist` array.

In [ ]:
len(result.states)

In [ ]:
result.states[-1]  # the finial state

You can visualize the time evolution of the state. `anim_matrix_histogram` visualizes the elements of it. The animation below shows that the state changes periodically and that the coefficient of  $\left|1\right>$ is real and that of $\left|0\right>$ is imaginary.

In [ ]:
fig, ani = anim_matrix_histogram(result, limits=[0, 1],
                                 bar_style='abs', color_style='phase')
# close an auto-generated plot and animation
plt.close()
ani

### Expectation values

The expectation values of an operator given a state vector or density matrix (or list thereof) can be calculated using the `expect` function.

In [ ]:
expect(sigmaz(), result.states[-1])

In [ ]:
expect(sigmaz(), result.states)

In [ ]:
fig, axes = plt.subplots(1, 1)

axes.plot(tlist, expect(sigmaz(), result.states))

axes.set_xlabel(r"$t$", fontsize=20)
axes.set_ylabel(r"$\left<\sigma_z\right>$", fontsize=20);

*If we are only interested in expectation values*, we could pass a list of operators to the `mesolve` function that we want expectation values for, and have the solver compute then and store the results in the `Odedata` class instance that it returns.

For example, to request that the solver calculates the expectation values for the operators $\sigma_x, \sigma_y, \sigma_z$:

In [ ]:
result1 = mesolve(H, psi0, tlist, [], [sigmax(), sigmay(), sigmaz()], options={"store_states": True})

In [ ]:
result1

In [ ]:
# result1.states[-1] will give an error.
# result1 = mesolve(H, psi0, tlist, [], [sigmax(), sigmay(), sigmaz()])
# -->  State not saved.

# If wanna have result1.states[-1],
# result1 = mesolve(H, psi0, tlist, [], [sigmax(), sigmay(), sigmaz()], options={"store_states": True})

In [ ]:
# result1.expect[0]

In [ ]:
# result1.expect[2]

In [ ]:
# result1.expect[1]

Now the expectation values are available in `result.expect[0]`, `result.expect[1]`, and `result.expect[2]`:

In [ ]:
fig, axes = plt.subplots(1, 1)

axes.plot(tlist, result1.expect[2], label=r"$\left<\sigma_z\right>$")
axes.plot(tlist, result1.expect[1], label=r"$\left<\sigma_y\right>$")
axes.plot(tlist, result1.expect[0], label=r"$\left<\sigma_x\right>$")

axes.set_xlabel(r"$t$", fontsize=20)
axes.legend(loc="best");

In [ ]:
print(tlist)

In [ ]:
import matplotlib.pyplot as plt

# 可视化 ⟨σₓ⟩, ⟨σᵧ⟩, ⟨σ_z⟩ 随时间的演化
plt.figure(figsize=(7, 6))
plt.plot(tlist, result1.expect[0], label=r'$\langle \sigma_x \rangle$')
plt.plot(tlist, result1.expect[1], label=r'$\langle \sigma_y \rangle$')
plt.plot(tlist, result1.expect[2], label=r'$\langle \sigma_z \rangle$')


plt.xlabel("Time")
plt.ylabel("Expectation values")
plt.title("Bloch vector components over time")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Dissipative dynamics

To add dissipation to a problem, all we need to do is to define a list of collapse operators to the call to the `mesolve` solver.

`A collapse operator` is an operator that *describes how the system is interacting with its environment*.

For example, consider a quantum harmonic oscillator with Hamiltonian

$H = \hbar\omega a^\dagger a$

and which loses photons to its environment with a relaxation rate $\kappa$. The collapse operator that describes this process is

$\sqrt{\kappa} a$

since $a$ is the photon annihilation operator of the oscillator.

To program this problem in QuTiP:

In [ ]:
w = 1.0  # oscillator frequency
kappa = 0.1  # relaxation rate
a = destroy(10)  # oscillator annihilation operator
rho0 = fock_dm(10, 5)  # initial state, fock state with 5 photons
H = w * a.dag() * a  # Hamiltonian

# A list of collapse operators
c_ops = [np.sqrt(kappa) * a]

In [ ]:
tlist = np.linspace(0, 50, 100)

# request that the solver return the expectation value
# of the photon number state operator a.dag() * a
result = mesolve(H, rho0, tlist, c_ops, [a.dag() * a],
                 options={"store_states": True})

In [ ]:
fig, axes = plt.subplots(1, 1)
axes.plot(tlist, result.expect[0])
axes.set_xlabel(r"$t$", fontsize=20)
axes.set_ylabel(r"Photon number", fontsize=16);

`anim_fock_distribution` enables you to visualize the probability distribution at each time.

In [ ]:
fig, ani = anim_fock_distribution(result)
# close an auto-generated plot and animation
plt.close()
ani

### Installation information

In [ ]:
about()